In [ ]:
import azureml.core
from azureml.core import Workspace, Dataset, Datastore, Environment
from azureml.core import Experiment
import azureml.interpret
from azureml.widgets import RunDetails

import pandas as pd
import numpy as np

print("SDK version:", azureml.core.VERSION)

In [ ]:
# Get the Workspace object from Azure
from azureml.core.authentication import InteractiveLoginAuthentication

# You can find tenant id under azure active directory->properties
tenant_id = '198c7d8c-e010-45ce-a018-ec2d9a33f58f'
ia = InteractiveLoginAuthentication(tenant_id=tenant_id)
ws_name = 'automlbook'
subscription_id = '4d278f3d-b4fd-4fa2-86b6-d34b96bc888f'
resource_group = 'Foxy_Resources'
ws = Workspace.get(name=ws_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,
                   auth=ia)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

In [ ]:
# (Print statements to list available environments)
# envs = Environment.list(workspace=ws)

# for env in envs:
    # if env.startswith("AzureML"):
        # print("Name",env)
        # if None != envs[env].python.conda_dependencies:
            # print("packages", envs[env].python.conda_dependencies.serialize_to_string())


In [ ]:
# Create datastore, try getting datastore via Workspace object
datastore = Datastore.get_default(ws)
datastore_name = 'workspaceblobstore'
datastore = Datastore.get(ws, datastore_name)

In [ ]:
# Create a dataset from the datastore of the Workspace
dataset_name = 'automlbook Titanic Training Data A'
# dataset = Dataset.get_by_name(ws, dataset_name)
dataset = Dataset.get_by_name(ws, dataset_name, version = 'latest')
dataset_columns = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']

# Show a sample of the data in the dataset
dataset.take(10).to_pandas_dataframe()

# Turn Dataset into Pandas Dataframe, it is to be preprocessed
df = dataset.to_pandas_dataframe()

In [ ]:
# (First way I did this, commented out) Preprocess numeric columns

# from sklearn.preprocessing import StandardScaler

# df_column_names = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']
# df_numeric_column_names = ['Age', 'Fare']

# # BEGIN Create dataframe with numeric columns so that it contains all numbers that are preprocessed...

# # For int column Age, Impute NaN numeric values, and Remove outliers
# print('Before Removing outliers or Imputing null values, df[Age]: ', df['Age'])
# ageMedian = np.nanmedian(df['Age'])
# print('ageMedian: ', ageMedian)
# df['Age'] = np.where(np.isnan(df['Age']), ageMedian, df['Age'])
# print('Before Removing outliers and after Imputing null values, df[Age]: ', df['Age'])

# # Calculate 3STD and Mean for Age
# ageThreeSD = np.std(df['Age']) * 3
# ageMean = np.mean(df['Age'])
# ageOutlierThreshold = round(ageThreeSD + ageMean)
# print('Age Outlier Threshold: ', ageOutlierThreshold)

# # Remove Outliers by replacing all values above Threshold (3STD + Mean) with Threshold Value
# df['Age'] = df['Age'].mask(df['Age'] > ageOutlierThreshold, ageOutlierThreshold)
# print('After Removing outliers and Imputing null values, df[Age]: ', df['Age'])

# # Copy df, keeping only Age column, set type of this df copy to float
# df_age_column = pd.DataFrame(df['Age'], columns=['Age'])


# # Copy df, keeping only float numeric columns, set type of this df copy to float
# df_float_column_names = ['Fare']
# print('df_float_column_names: ', df_float_column_names)
# df_float_columns = pd.DataFrame(df[df_float_column_names], dtype=np.float, columns=df_float_column_names)

# # Concatenate the numeric Data Frames to scale them
# print('Before concatenation to df_numeric_columns, df[Age]: ', df['Age'])
# print('Before concatenation to df_numeric_columns, df_age_column: ', df_age_column)
# df_numeric_columns = pd.concat([df_age_column, df_float_columns], keys=df_numeric_column_names, axis=1)
# print('concatenated df_numeric_columns: ', df_numeric_columns)


# # Use StandardScaler or MinMaxScaler on Numeric/Non-Categorical columns split
# scaler = StandardScaler().fit(df_numeric_columns)
# print('scaler.mean_: ', scaler.mean_)
# print('scaler.scale: ', scaler.scale_)

# df_scaled_numeric_columns =  pd.DataFrame(scaler.transform(df_numeric_columns), columns=df_numeric_column_names)
# print('df_scaled_numeric_columns: ', df_scaled_numeric_columns)
# # Scaled data should have zero mean and unit variance, check with these prints:
# print('df_scaled_numeric_columns.mean(axis=0): ', df_scaled_numeric_columns.mean(axis=0))
# print('df_scaled_numeric_columns.std(axis=0)', df_scaled_numeric_columns.std(axis=0))

In [ ]:
# (First way I did this, commented out) Preprocess categorical columns

# from sklearn.preprocessing import OneHotEncoder

# # BEGIN Create dataframe with categorical columns so that it contains all categorical data that is preprocessed...

# # Copy df, keeping only categorical columns, and one-hot encode them
# df_categorical_column_names_raw = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked']
# # print('df_categorical_column_names_raw: ', df_categorical_column_names_raw)
# df_categorical_columns = pd.DataFrame(df[df_categorical_column_names_raw], dtype=np.str, columns=df_categorical_column_names_raw)
# # print('df_categorical_columns: ', df_categorical_columns)
# encoder = OneHotEncoder(drop='first', handle_unknown='error', sparse=False, ).fit(df_categorical_columns)
# # print('encoder.categories: ', encoder.categories)
# df_encoded_categorical_columns = pd.DataFrame(encoder.transform(df_categorical_columns))
# df_encoded_categorical_columns.columns = encoder.get_feature_names(df_categorical_column_names_raw)
# # print('df_encoded_categorical_columns: ', df_encoded_categorical_columns)
# #   By default, the values each feature can take is inferred automatically from the dataset and can be found in the categories_ attribute:


# # Combine the numeric DF with the categorical DF
# dfs = [df['Survived'], df_scaled_numeric_columns, df_encoded_categorical_columns]
# # print('Before concatenation to dfTyped, df[\'Survived\']: ', df['Survived'])
# # print('Before concatenation to dfTyped, df[Age]: ', df['Age'])
# # print('Before concatenation to dfTyped, df_numeric_columns: ', df_numeric_columns)
# # print('Before concatenation to dfTyped, df_scaled_numeric_columns: ', df_scaled_numeric_columns)
# ??? shouldn't column names be passed in to the keys list here?
# dfTyped = pd.concat(dfs, axis=1, keys=['Survived', *df_scaled_numeric_columns, *df_encoded_categorical_columns])
# # print('dfTyped: ', dfTyped)
# # print('dfTyped[Age]: ', dfTyped['Age'])

#         # - With sklearn.preprocessing, preprocess your Dataframes before training model in the Python Script
#         #     - [Guide at SciKit Learn site](https://scikit-learn.org/stable/modules/preprocessing.html)
#         #     - Use OneHotEncoder
#         #     - Use StandardScaler or  MinMaxScaler while you're at it
#         #     - Don't worry about any other preprocessing to just get the training working
#         #     - Strategy:
#         #         - d Split dataframe into Numeric/Non-Categorial and Non-Numeric/Categorial columns
#         #             - ! Use StandardScaler or MinMaxScaler on Numeric/Non-Categorical columns split
#         #             - d Use OneHotEncoder on Non-Numeric/Categorical columns split

# # Initial Data Frame is now preprocessed in dfPreprocessed
# dfPreprocessed = dfTyped
# # print('dfPreprocessed: ', dfPreprocessed)

# # Split DataFrame for training now that it is pre-processed
# target_column_name = 'Survived'
# df_x = dfPreprocessed.drop([target_column_name], axis=1)
# df_y = dfPreprocessed[target_column_name]
# # print("See df_x", df_x)
# print("See df_y", df_y)

In [ ]:
# Partition Dataframe to get one for Numeric columns and one for Categorical columns

df_column_names = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']

##### BEGIN Create dataframe with numeric columns so that it contains all numbers that are preprocessed...
df_numeric_column_names = ['Age', 'Fare']
# Copy df, keeping only float numeric columns, set type of this df copy to float
df_float_column_names = ['Fare']
df_float_columns = pd.DataFrame(df[df_float_column_names], dtype=np.float, columns=df_float_column_names)
# # Copy df, keeping only integer Age column to leave as an integer
df_integer_column = pd.DataFrame(df['Age'], columns=['Age'])
# Concatenate the numeric DataFrames
df_numeric_columns = pd.concat([df_integer_column, df_float_columns], axis=1)


##### BEGIN Create dataframe with categorical columns so that it contains all categorical data that is preprocessed...
df_categorical_column_names = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked']
# Copy df, keeping only categorical columns
df_categorical_columns = pd.DataFrame(df[df_categorical_column_names], dtype=np.str, columns=df_categorical_column_names)

print('concatenated df_numeric_columns: ', df_numeric_columns)
print('df_categorical_columns: ', df_categorical_columns)

In [ ]:
# Create preprocessor to preprocess numeric and categorical columns (with Transfomer API via ColumnTransformer, including creation of an Explainer)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', one_hot_encoder)])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, df_numeric_column_names),
        ('cat', categorical_transformer, df_categorical_column_names)])

In [ ]:
# Append classifier to preprocessing pipeline (Then we have a full prediction pipeline)
from sklearn.tree import DecisionTreeClassifier

regressor = DecisionTreeClassifier()

classifier_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', regressor)])
# The classifier_pipeline is later de-pickled and used in the Experiment Run's python script

# Now we have a full prediction pipeline.
print(classifier_pipeline.__dict__)
# Note that you may access classifier_pipeline ColumnTransformers as follows...
# print(classifier_pipeline['preprocessor'].transformers[1][1][1].get_feature_names(df_categorical_column_names))

In [ ]:
# Pickle the SciKit Learn Pipeline to pass it to the script
# Link about pickling:
# https://codefather.tech/blog/python-pickle/#:~:text=%20Python%20Pickle%3A%20Serialize%20Your%20Objects%20%20,The%20pickle%20module%20also%20allows%20to...%20More%20
import pickle

pickled_pipeline = pickle.dumps(classifier_pipeline)
# print(pickle.loads(pickled_pipeline))

with open('./scripts/resources/classifier_pipeline.pickle', 'wb') as file:
    pickle.dump(classifier_pipeline, file)

# You may copy this to the script and Unpickle the SciKit Pipline (that performs Transformation and Model Training)
# with open('classifier_pipeline.pickle', 'rb') as file:
#     unpickled_pipeline = pickle.load(file)
#     print(unpickled_pipeline)

In [ ]:
# (Does nothing at the moment) Serialize the SciKit Learn Pipeline to pass it to the script (with simple_python_to_json)
# I Quit this to use pickling just to get this working and avoid maintaining simple_python_to_json for now...
#       lesson learned, 
#       ideally the code would serialize/deserialize every single Sklearn object type...
# TODO? maybe add to simple_python_to_json get this cell working?
# You may use code within to help you serialize
#      (Contain the code behind the 'simple_python_to_json' reference )
# https://robotfantastic.org/serializing-python-data-to-json-some-edge-cases.html
# https://cmry.github.io/notes/serialize
# import json
# import simple_python_to_json
# def serialize_dict(in_dictionary, name):
#     print(classifier_pipeline.__dict__)
#     for k, v in in_dictionary.items():
#         print("k,v is: ", k, ", ", v)
#         print(simple_python_to_json.serializer.json_to_data(simple_python_to_json.serializer.data_to_json(v)))
#         in_dictionary[k] = simple_python_to_json.data_to_json(v)
#     json.dump(in_dictionary, open(name + '.json', 'w'))

# TODO? maybe finish changes to serializer.py to serialize this
# serialize_dict(classifier_pipeline.__dict__, 'classifier_pipeline')

# TODO? maybe copy below to script, make todo to get it working in script
# Load the json and deserialize:

# def deserialize(class_init, attr):
#     for k, v in attr.items():
#         setattr(class_init, k, sr.json_to_data(v))
#     return class_init

# Load and deserialize the json:
# deserialized_classifier_pipeline = deserialize(Pipeline(), json.load(open('classifier_pipeline.json'))

In [ ]:
# Combine the numeric DF with the categorical DF
# print("df['Survived'] is ", df['Survived'])
# print("df_numeric_columns is ", df_numeric_columns)
# print("df_numeric_columns.columns is ", df_numeric_columns.columns)
# print("df_categorical_columns is ", df_categorical_columns)
# print("df_categorical_columns.columns is ", df_categorical_columns.columns)

# Concatenate dfs to get DataFrame of all columns to submit to the classifier_pipeline
dfs = [df['Survived'], df_numeric_columns, df_categorical_columns]
# print("dfs is" + str(dfs))
# print('Before concatenation to dfTyped, df[\'Survived\']: ', df['Survived'])
# print('Before concatenation to dfTyped, df_numeric_columns: ', df_numeric_columns)
dfTyped = pd.concat(dfs, axis=1)
print('dfTyped: ', dfTyped)

In [ ]:
# Split pre-transformation Data Frame into feature/target columns
target_column_name = 'Survived'
df_x = dfTyped.drop([target_column_name], axis=1)
df_y = dfTyped[target_column_name].to_frame()
# print("See df_x", df_x)
print("See df_y", df_y)

In [ ]:
# (Validate the Pipeline will function once passed to the script)

# Get the preprocessed Data Frame columns in a list
one_hot_encoder.fit(df_categorical_columns)
# Get new One Hot Encoded column names
# df_encoded_categorical_column_names = one_hot_encoder.get_feature_names(df_categorical_column_names)
# print(str(df_encoded_categorical_column_names))

# (fail to) Get the preprocessed Data Frame
# # dfPreprocessed = preprocessor.fit_transform(df_x_pre_transformation, df_y_pre_transformation)

# Attempt to fit the preprocessor to the data, to validate it will function inside the script it is passed to
preprocessor.fit(df_x, df_y)

In [ ]:
# Register Pandas Dataframe of base df_x and df_y
Dataset.Tabular.register_pandas_dataframe(df_x, datastore, "Titanic Feature Column Data for train_test_split usage (LocalDocker notebook)")
Dataset.Tabular.register_pandas_dataframe(df_y, datastore, "Titanic Target Column Data for train_test_split usage (LocalDocker notebook)")

In [ ]:
# Split data into training and test data, register the resulting Datasets with Azure
from sklearn.model_selection import train_test_split

# Split the data
# What you need to pass to train_test_split...
# ... I need X and Y dataframe, X just with target missing, Y just with target column present
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,
                                                    test_size=0.2,
                                                    random_state=0)
# print("See y_test", y_test)
print("See y_test.columns.tolist()", str(y_test.columns.tolist()))
# print("See y_test.values.tolist() to pass as true_ys to ExplanationClient.upload_model_explanation()", y_test.values.tolist())
# print("See y_test.values.ravel() to pass as true_ys to ExplanationClient.upload_model_explanation()", y_test.values.ravel())
# print("See y_test.values.tolist().flatten() to pass as true_ys to ExplanationClient.upload_model_explanation()", y_test.values.flatten())

# Register the splits
X_train_registered_name = "Titanic Feature Column Data for training (LocalDocker notebook)"
X_test_registered_name = "Titanic Feature Column Data for testing (LocalDocker notebook)"
y_train_registered_name = "Titanic Target Column Data for training (LocalDocker notebook)"
y_test_registered_name = "Titanic Target Column Data for testing (LocalDocker notebook)"
Dataset.Tabular.register_pandas_dataframe(X_train, datastore, X_train_registered_name)
Dataset.Tabular.register_pandas_dataframe(X_test, datastore, X_test_registered_name)
Dataset.Tabular.register_pandas_dataframe(y_train, datastore, y_train_registered_name)
Dataset.Tabular.register_pandas_dataframe(y_test, datastore, y_test_registered_name)
xTrainTestYTrainTest = [X_train_registered_name, X_test_registered_name, y_train_registered_name, y_test_registered_name]

In [ ]:
# Save feature names and create TabularExplainer with them
features=[*df_numeric_column_names, *df_categorical_column_names]

In [ ]:
# Encode Experiment script arguments list into a string like '["a","b"]'

# Encode numeric column names list
temp_column_names = df_numeric_column_names.copy()
for x in range(len(temp_column_names)):
        temp_column_names[x] = '"{}"'.format(temp_column_names[x])
numericFeatureNamesEncoded = "[{}]".format(",".join(temp_column_names))
# print("numericFeatureNamesEncoded:", numericFeatureNamesEncoded)

# Encode categoric column names list
temp_column_names = df_categorical_column_names.copy()
for x in range(len(temp_column_names)):
        temp_column_names[x] = '"{}"'.format(temp_column_names[x])
categoricFeatureNamesEncoded = "[{}]".format(",".join(temp_column_names))
# print("categoricFeatureNamesEncoded:", categoricFeatureNamesEncoded)

# Encode split dataset names list
for x in range(len(xTrainTestYTrainTest)):
        xTrainTestYTrainTest[x] = '"{}"'.format(xTrainTestYTrainTest[x])
xTrainTestYTrainTestEncoded = "[{}]".format(",".join(xTrainTestYTrainTest))
print("splitDatasetNamesEncoded:", xTrainTestYTrainTestEncoded)

In [ ]:
# Set Local Docker Environment up (with System Managed Dependencies, via Conda)
#
# Learn about Environment and how to use a Docker Environment here:
#       https://docs.microsoft.com/en-us/azure/machine-learning/concept-environments
#       https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py
#       ! IMPORTANT: https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/
#
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

docker_env = Environment("docker-env")
# Editing a run configuration property on-fly.
docker_env.python.user_managed_dependencies = False
# Use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param.
#           https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfig.dockerconfiguration?view=azure-ml-py
docker_config = DockerConfiguration(use_docker=True)
print("initial base image from base docker-env Environment: ", docker_env.docker.base_image)

# Specify docker steps as a string. 
dockerfile = r"""
FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220314.v1

ARG DEBIAN_FRONTEND=noninteractive
RUN apt-get update -y && apt-get upgrade -y &&\
    apt-get install -y build-essential \
                       cmake \
                       curl \
                       gfortran \
                       git \
                       jupyter \
                       libatlas-base-dev \
                       libblas-dev \
                       libbz2-dev \
                       libffi-dev \
                       libgdbm-dev \
                       liblapack-dev \
                       liblzma-dev \
                       libncurses5-dev \
                       libncursesw5-dev \
                       libreadline-dev \
                       libsqlite3-dev \
                       libssl-dev \
                       libxml2-dev \
                       libxmlsec1-dev \
                       llvm \
                       lzma \
                       lzma-dev \
                       make \
                       tcl-dev \
                       tk-dev \
                       wget \
                       xz-utils \
                       zlib1g-dev

RUN echo "Hello from custom container!" > ~/hello.txt
"""

# NOTE: you can pass Dockerfile string to docker build command via stdin like this:
#
# sudo docker build -t myimage:latest -<<EOF
# FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220314.v1
# RUN echo "hello world"
# EOF


# Set base image to None, because the image is defined by dockerfile.
docker_env.docker.base_image = None
# Use the Dockerfile string and build image based on it with this code from above (move it down here)
docker_env.docker.base_dockerfile = dockerfile

#       For help, try reading this
#                    - https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/#custom-docker-image--dockerfile
#       May also try using other base Docker images from these container registries
#                - https://github.com/microsoft/containerregistry
#                - https://github.com/Azure/AzureML-Containers
# env.docker.base_image = '<image-name>'
# env.docker.base_image_registry.address = '<container-registry-address>'
# env.docker.base_image_registry.username = '<acr-username>'
# env.docker.base_image_registry.password = os.environ.get("CONTAINER_PASSWORD")
#
# TODO? use this code to set username and password:
# # Retrieve username and password from the workspace key vault
#       env.docker.base_image_registry.username = ws.get_default_keyvault().get_secret("username")  
#       env.docker.base_image_registry.password = ws.get_default_keyvault().get_secret("password")


# Specify conda dependencies with scikit-learn
conda_packages = ['pip',
                  'pyspark',
                  'scikit-learn'
                 ]
pip_packages =   ['azureml.interpret',
                  'azureml-dataset-runtime',
                  'jinja2',
                  'MarkupSafe',
                  'raiwidgets'
                 ]
                 
condaDependencies = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
docker_env.python.conda_dependencies = condaDependencies
# NOTE: can set this:
#           docker_env.python.interpreter_path = "/opt/miniconda/bin/python"

# Register the Docker Environment and build the Docker image locally
registered_docker_env = docker_env.register(ws)
print(registered_docker_env)
registered_docker_env.build_local(ws, useDocker=False, pushImageToWorkspaceAcr=True)

In [ ]:
# Prepare to run Training Experiment in Docker Environment (With Docker running on local device)

from azureml.core import ScriptRunConfig
import datetime

# Experiment
experiment_name = 'LocalDocker_Training_AutoML'
experiment = Experiment(workspace=ws, name=experiment_name)

# Define Compute Cluster to use
compute_target = 'local'
source_directory = './scripts'
script_name = 'localDockerTrainingAutoML.py'
dataset_name = 'automlbook Titanic Training Data A'
# set output file name like 'DecisionTreeClassifier_Titanic_LocalDocker-2022-04-17 21:40:36.114550.pkl'
suffix = 'local-' + str(datetime.datetime.now())
suffix = suffix.replace(' ', '_') # Clean up datetimestamp
suffix = suffix.replace(':', '-') 
out_model_file_name = 'DecisionTreeClassifier_Titanic_LocalDocker_{}.pkl'.format(suffix)
# set output file name like 'DecisionTreeClassifier_Titanic_LocalDocker-2022-04-17 21:40:36.114550.pkl'

script_arguments = [
"--tenant-id", tenant_id,
"--ws-name", ws_name,
"--subscription-id", subscription_id,
"--resource-group", resource_group,
"--datastore-name", datastore_name,
"--out-model-file-name", out_model_file_name,
"--numeric-feature-names", numericFeatureNamesEncoded,
# Pass list encoded as a comma-separated string, containing the raw feature names
# like '["a","b"]'
"--categoric-feature-names", categoricFeatureNamesEncoded,
# Pass list encoded as a comma-separated string, containing the name of each dataset (X_train, X_test, y_train, y_test)
# like '["a","b"]'
"--x-train-test-y-train-test", xTrainTestYTrainTestEncoded 
]
print("ScriptRunConfig arguments: ", script_arguments)
scriptRunConfig = ScriptRunConfig(
        source_directory=source_directory,
        script=script_name,
        arguments=script_arguments,
        environment=docker_env,
        docker_runtime_config=docker_config)

In [ ]:
# Submit Experiment Run to Docker environment
#
# (see more on use of Docker environment: 
#   https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/train-on-local/train-on-local.ipynb
# )
import subprocess
# TODO? Add minconda bin to path within docker container? "/home/johna/miniconda3/bin:"

subprocess.run("whoami", shell=True)

# import getpass
# If you need, you can get a password from user input (Notebook pauses to show a prompt here)
# password = getpass.getpass()

scriptRunConfig.run_config.environment = docker_env
# Check if Docker is installed and Linux containers are enabled
if subprocess.run("docker -v", shell=True).returncode == 0:
    subprocess.run("service docker status", shell=True)


    # This StackOverflow page will help you run the docker commands with sudo if that is necessary for you: 
    # https://askubuntu.com/questions/155791/how-do-i-sudo-a-command-in-a-script-without-being-asked-for-a-password
    #
    # (NOTE: These snippets from the link will allow you to have the docker commandsd run with sudo)
    #
    # Create script with the command to run with sudo, for example:
    # docker system info
    #
    # Run sudo chown and chmod commands to grant access of the file to root 
    # sudo chown root:root ~/docker_system_info.sh
    # sudo chmod 700 ~/docker_system_info.sh
    #
    # Run sudo visudo and insert a line below the line `%sudo   ALL=(ALL:ALL) ALL`
    # [username]  ALL=(ALL) NOPASSWD: /home/[username]/docker_system_info.sh
    #
    # Then call the python script with your python subprocess command
    # p = subprocess.run('sudo ~/docker_system_info.sh', shell=True)


    out = subprocess.check_output('sudo ~/docker_system_info.sh', shell=True).decode('ascii')
    if not "OSType: linux" in out:
        print("Switch Docker engine to use Linux containers.")
    else:
        # TODO get this experiment run to work, it is hanging, perhaps I need apt dependencies added to Dockerfile string I passed(??)
        # TODO update: still not getting any logs past "nvidia-docker is installed on the target. Using nvidia-docker for docker operations"
        # TODO update: perhaps it is the user that is being used to run docker command?
        #                   (perhaps allow docker to run by johna user normally, without any black magic)
        # TODO!!!!!! update: perhaps add authentication for getting the image from whatever registry it is being gotten from
        #                   See: https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/
        #
        AutoML_run = experiment.submit(scriptRunConfig)
        RunDetails(AutoML_run).show()
else:
    print("Docker engine is not installed.")

In [ ]:

# TODO DELETE THIS BACKUP CODE
# Submit Experiment Run to Docker environment
#
# (see more on use of Docker environment: 
#   https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/train-on-local/train-on-local.ipynb
# )
# import subprocess
# # TODO? Add minconda bin to path within docker container? "/home/johna/miniconda3/bin:"

# subprocess.run("whoami", shell=True)

# import getpass
# import os
# # TODO run failing commands with sudo like this.) Get password from user input (Notebook pauses to show a prompt here)
# password = getpass.getpass()

# # command = "sudo -S docker system info" # can be any command but don't forget -S as it enables input from stdin
# # os.popen(command, 'w').write(password+'\n') # newline char is important otherwise prompt will wait for you to manually perform newline

# scriptRunConfig.run_config.environment = docker_env
# # Check if Docker is installed and Linux containers are enabled
# if subprocess.run("docker -v", shell=True).returncode == 0:
#     subprocess.run("service docker status", shell=True)
#     # TODO FIX THIS perhaps have docker run as a user, or can I pass a username/password?
#     command = "sudo -S docker system info" # can be any command but don't forget -S as it enables input from stdin
#     print(os.popen(command, 'w').write(password+'\n')) # newline char is important otherwise prompt will wait for you to manually perform newline
#     out = subprocess.check_output("sudo docker system info", shell=True).decode('ascii')
#     print("out: ", out)
#     if not "OSType: linux" in out:
#         print("Switch Docker engine to use Linux containers.")
#     else:
#         AutoML_run = experiment.submit(scriptRunConfig)
#         RunDetails(AutoML_run).show()
# else:
#     print("Docker engine is not installed.")

In [ ]:
# (While Experiment script runs, Validate the Engineered Feature Explanation will function inside the script)

# Split training features into numeric and categoric dataframes
numeric_X_test = pd.DataFrame(X_test[df_numeric_column_names], dtype=np.str, columns=df_numeric_column_names)
categoric_X_test = pd.DataFrame(X_test[df_categorical_column_names], dtype=np.str, columns=df_categorical_column_names)
# Fit and Run the numeric and categoric ColumnTransformers on the split dataframes to perform feature engineering
preprocessor.transformers[0][1].fit(numeric_X_test)
preprocessor.transformers[1][1].steps[0][1].fit(categoric_X_test)
numeric_X_test_preprocessed = preprocessor.transformers[0][1].transform(numeric_X_test)
numeric_X_test_preprocessed = pd.DataFrame(numeric_X_test_preprocessed, dtype=np.float, columns=df_numeric_column_names)
categoric_X_test_preprocessed = preprocessor.transformers[1][1].steps[0][1].transform(categoric_X_test)
# Fit OneHotEncoder
preprocessor.transformers[1][1].steps[1][1].fit(categoric_X_test_preprocessed)
# Get new One Hot Encoded column names
print(categoric_X_test_preprocessed)
print(df_categorical_column_names)
df_encoded_categorical_column_names = preprocessor.transformers[1][1].steps[1][1].get_feature_names(df_categorical_column_names)
print("df_encoded_categorical_column_names", df_encoded_categorical_column_names)
# Transform categoric, null-imputed features with fitted OneHotEncoder
categoric_X_test_preprocessed = preprocessor.transformers[1][1].steps[1][1].transform(categoric_X_test_preprocessed)
# Turn preprocessed categoric features into a DataFrame
categoric_X_test_preprocessed = pd.DataFrame(categoric_X_test_preprocessed, dtype=np.float64, columns=df_encoded_categorical_column_names)

# Combine the numeric DF with the categorical DF to submit to the classifier_pipeline
X_test_preprocessed_list = [numeric_X_test_preprocessed, categoric_X_test_preprocessed]
X_test_preprocessed = pd.concat(X_test_preprocessed_list, axis=1)


# Save engineered features' names to create TabularExplainer with them
engineeredFeatures=[*df_numeric_column_names, *df_encoded_categorical_column_names]

# Fit the model
classifier_pipeline.steps[-1][1].fit(X_test_preprocessed, y_test)
# Explain in terms of engineered features
# NOTE: classifier_pipeline.steps[-1][1] contains the Model
# NOTE: "features" and "classes" fields are optional for TabularExplainers
from interpret.ext.blackbox import TabularExplainer
engineered_explainer = TabularExplainer(classifier_pipeline.steps[-1][1],
                                     initialization_examples=X_test_preprocessed,
                                     features=engineeredFeatures)
# Explain results with this Explainer and upload the Explanation...

# Get Global Explanations of raw features, global as in 'of total data'...
# You can use the training data or the test data here, but test data would allow you to use Explanation Exploration
# print("X_test, line value before engineered_explainer.explain_global: \n" + str(X_test))
global_explanation = engineered_explainer.explain_global(X_test_preprocessed, y_test)
# If you used the PFIExplainer in the previous step, use the next line of code instead
# global_explanation = engineered_explainer.explain_global(X_test, true_labels=y_train)
# Sorted feature importance values and feature names
sorted_global_importance_values = global_explanation.get_ranked_global_values()
sorted_global_importance_names = global_explanation.get_ranked_global_names()
globalFeatureExplanations = dict(zip(sorted_global_importance_names, sorted_global_importance_values))
print('globalFeatureExplanations: ', globalFeatureExplanations)
# Alternatively, you can print out a dictionary that holds the top K feature names and values
print('global_explanation.get_feature_importance_dict(): ', global_explanation.get_feature_importance_dict())

# Upload the explanation in terms of engineered features
from azureml.interpret import ExplanationClient
client = ExplanationClient.from_run(AutoML_run)
client.upload_model_explanation(global_explanation, true_ys=y_test.values.ravel(), comment='global explanation: test dataset features, engineered')

In [ ]:
# Register Model from the AutoML_run
description = "Best LocalDocker AutoML Regression Run using Titanic Sample Data."
tags = {
  "project" : "LocalDocker Training AutoML", 
  "creator": "fox", 
  "task": "classification", 
  "dataset": "automlbook Titanic Training Data A", 
  "metric": "normalized_root_mean_squared_error"
}

# Attempt to register model once output model file is available
from azureml.core import Model
import sklearn
import time
# TODO get this working
AutoML_run.wait_for_completion(show_output=True)
while True:
  try:
    AutoML_run.register_model(model_path='./outputs', model_name=out_model_file_name, description=description, tags=tags,
                            model_framework=Model.Framework.SCIKITLEARN, # Framework used to create the model.
                            model_framework_version=sklearn.__version__)  # Version of scikit-learn used to create the model.)
    break
  except:
      print ("encountered exception registering model output file, waiting and trying again...") 
      time.sleep(60)
# Set output file name like 'DecisionTreeClassifier_Titanic_LocalDocker-2022-04-17 21:40:36.114550.pkl'

In [ ]:
# (Now done in script instead of this Notebook) Upload global model explanation data...
# from azureml.interpret import ExplanationClient

# The explanation can then be downloaded on any compute
# Multiple explanations can be uploaded
# print("y_test value the line before client.upload_model_explanation(): \n" + str(y_test))
# print("y_test.values.ravel() value passed as true_ys to client.upload_model_explanation(): \n" + str(y_test.values.ravel()))
# client = ExplanationClient.from_run(AutoML_run)
# client.upload_model_explanation(global_explanation, true_ys=y_test.values.ravel(), comment='global explanation: all features')

# Or you can only upload the explanation object with the top k feature info with this...
# client.upload_model_explanation(global_explanation, top_k=2, comment='global explanation: Only top 2 features')
# END Upload global model explanation data...

In [ ]:
# For the Experiment Run that was ran, Get Global Explanations (Downloaded with the Experiment Run object!)
from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(AutoML_run)

# Get model explanation data
# TODO? There should be a way to download_model_explanation by comment instead of id.
print(client.list_model_explanations())
# NOTE: this step will fail until you copy the explanation_id values from the printed list of explanations
engineered_global_explanation_test = client.download_model_explanation(explanation_id='2ef948ba-9b01-453e-80da-f1c498022e5b')
engineered_global_explanation_train = client.download_model_explanation(explanation_id='846c159e-5ee7-4c79-8994-8757055de83a')
# TODO? Do something with the Engineered Feature explanations here
global_explanation = client.download_model_explanation(explanation_id='6eac21c3-891c-4d43-beae-62eb7b5b9965')



# Or only get the top k (e.g., 4) most important features with their importance values
# explanation = client.download_model_explanation(top_k=4)

global_importance_values = global_explanation.get_ranked_global_values()
global_importance_names = global_explanation.get_ranked_global_names()
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

In [ ]:
# Download the Model from AzureML and Visualize Explanations with it
from raiwidgets import ExplanationDashboard
from azureml.core.model import Model
import joblib
import jinja2

# print('Model.list(ws)', Model.list(ws))

# Download the Model from Azure

# Try 1: just Model constructor and joblib.load()
# downloaded_model = Model(ws, out_model_file_name)
# joblib.load(downloaded_model)

# Try 2: Use Model.get_model_path and joblib.load()
# remote_model_path = Model.get_model_path(out_model_file_name, _workspace=ws)
# downloaded_model = joblib.load(remote_model_path)

# Try 3: Use Model.download and joblib.load()
remote_model_obj = Model(ws, out_model_file_name)
print('Name:', remote_model_obj.name)
print('Version:', remote_model_obj.version)
remote_model_path = remote_model_obj.download(exist_ok = True)
downloaded_model = joblib.load(remote_model_path)

# BEGIN Access "Local Explanations", uncomment these lines if you want to do that here...
# (Local Explanation meaning "of individual predictions") 
# from interpret.ext.blackbox import TabularExplainer
# "features" and "classes" fields are optional
# explainer = TabularExplainer(downloaded_model,
                            #  X_train,
                            #  features=features)
#
# Get explanation for the first few data points in the train set
# local_explanation = explainer.explain_local(X_train[0:5])
# Sorted feature importance values and feature names
# sorted_local_importance_names = local_explanation.get_ranked_local_names()
# print('sorted_local_importance_names: ', sorted_local_importance_names)
# print('len(sorted_local_importance_names): ', len(sorted_local_importance_names))
# sorted_local_importance_values = local_explanation.get_ranked_local_values()
# print('sorted_local_importance_values: ', sorted_local_importance_values)
# print('len(sorted_local_importance_values): ', len(sorted_local_importance_values))
# COOL THING TO DO: Sometime could get local explanation of specific data points uploaded, downloaded, and visualized as well...
# END Access "Local Explanations"

# Visualize explanations
# Be sure to pass dataset=(train feature columns Dataframe) and true_y=(train predicted column Dataframe)
#
# TODO Get base statistics to appear in visualization's Model Performance tab: 
# TODO      (the accuracy, precision, f1 scores, false positive rates, false negative rates...)
#
# TODO? Show both raw and engineered features from raiwidgets ExplanationDashboard like this.
#
#       1) getting the raiwidgets thing working
#       2) see README at https://github.com/interpretml/interpret
ExplanationDashboard(global_explanation, downloaded_model, dataset=X_train, true_y=y_train)